# Import & Path

## Import

In [ ]:
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pickle
import random

## Path

In [ ]:
basePath = os.getcwd()
trainPath = os.path.join(os.getcwd(), "Images")
testPath = os.path.join(os.getcwd(), "Online-Test-sort")
xlsFile = os.path.join(os.getcwd(), "modelList.xlsx")
print(trainPath)
print(testPath)

# Default Configuration

In [ ]:
height = 128
width = 128
epochs = 50

# Score

## Create Model List

In [ ]:
def create_modellist(path):
    modelList = pd.DataFrame(columns=['model', 'accuracy', 'loss', 'val_accuracy', 'val_loss', 'test_accuracy', 'avg_precision', 'avg_recall', 'avg_f1_score'])
    
    for x in os.listdir(path):
        if x.endswith('.h5'):
            modelList = pd.concat([modelList, pd.DataFrame([[x.split('.')[0], 0, 0, 0, 0, 0, 0, 0, 0]], columns=['model', 'accuracy', 'loss', 'val_accuracy', 'val_loss', 'test_accuracy', 'avg_precision', 'avg_recall', 'avg_f1_score'])], ignore_index=True)
        elif x.endswith('.pickle'):
            with open(os.path.join(path, x), 'rb') as file_pi:
                data = pickle.load(file_pi)
                modelList.loc[modelList['model'] == x.split('.')[0], 'accuracy'] = data['accuracy'][-1]
                modelList.loc[modelList['model'] == x.split('.')[0], 'loss'] = data['loss'][-1]
                
                if 'val_accuracy' in data:
                    modelList.loc[modelList['model'] == x.split('.')[0], 'val_accuracy'] = data['val_accuracy'][-1]
                    modelList.loc[modelList['model'] == x.split('.')[0], 'val_loss'] = data['val_loss'][-1]

                # EvaluateModel(x.split('.')[0], data)
    
    return modelList

## Generate Test Data

In [ ]:
def data_generator(path, batch_size):
    test_datagen = ImageDataGenerator()
    batch_size = batch_size

    test_generator = test_datagen.flow_from_directory(
        path,
        target_size=(height, width),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=batch_size)

    labels = test_generator.classes
    filenames = test_generator.filenames
    nb_samples = np.ceil(len(filenames)/batch_size)

    return test_generator, nb_samples, labels

## Evaluate Model

In [ ]:
def confMatrix(path, modelName, labels, pred):
    cf = confusion_matrix(labels, pred)
    plt.clf()
    sns.heatmap(cf, annot=True, cmap="Blues")
    plt.savefig(os.path.join(path, (modelName + '-confusion-matrix.png')))

In [ ]:
def classificationReport(path, modelName, labels, pred):
    cr = classification_report(labels, pred, output_dict=True)

    df = pd.DataFrame(cr).transpose()
    df.to_excel(os.path.join(path, (modelName + '-classification-report.xlsx')))

    precicion = df.loc['macro avg', 'precision']
    recall = df.loc['macro avg', 'recall']
    f1_score = df.loc['macro avg', 'f1-score']

    return precicion, recall, f1_score

## Predict Test Data

In [ ]:
def predict(path, pathTest, modelList, max_accuracy, max_accuracy_model, max_accuracy_pred):
    plt.figure(figsize = (20,20))
    sns.set(font_scale=1.4)

    for x in modelList['model']:
        model = load_model(os.path.join(path, (x + '.h5')))
        config = model.get_config()
        config = config["layers"][0]["config"]["batch_input_shape"]
        
        if ((config[1] != height) or (config[2] != width)):
            print("Different input size!")
            print("Expected: {}x{}".format(config[1],config[2]))
            print("Given: {}x{}".format(height, width))
            print("Model name:{}".format(x))
            continue

        batch_size = x.split('-')[1]
        batch_size = int(batch_size)

        test_generator, nb_samples, labels = data_generator(pathTest, batch_size)

        if nb_samples is not None:
            predict_x = model.predict(test_generator, steps=nb_samples)
        else:
            predict_x = model.predict(test_generator)

        pred = np.argmax(predict_x, axis=1)

        print(x, config[2])

        accuracy = accuracy_score(labels, pred)
        print('Test Data accuracy: ', accuracy * 100)

        modelList.loc[modelList['model'] == x, 'test_accuracy'] = accuracy

        confMatrix(path, x, labels, pred)
        precicion, recall, f1_score = classificationReport(path, x, labels, pred)

        modelList.loc[modelList['model'] == x, 'avg_precision'] = precicion
        modelList.loc[modelList['model'] == x, 'avg_recall'] = recall
        modelList.loc[modelList['model'] == x, 'avg_f1_score'] = f1_score

        if accuracy > max_accuracy:
            max_accuracy = accuracy
            max_accuracy_model = x
            max_accuracy_pred = pred

    plt.close()

    print('Max accuracy: ', max_accuracy * 100)
    print('Max accuracy model: ', max_accuracy_model)

    return max_accuracy, max_accuracy_model, max_accuracy_pred, modelList

# Predict

In [ ]:
modelList = pd.DataFrame(columns=['model', 'accuracy', 'loss', 'val_accuracy', 'val_loss', 'test_accuracy', 'avg_precision', 'avg_recall', 'avg_f1_score'])
enhance_type = ['none', 'gaussian', 'sharpen', 'clahe', 'combo', 'robust']
# enhance_type = ['robust']
max_accuracy = 0
max_accuracy_model = ''
max_accuracy_pred = []

for x in enhance_type:
    pathModel = os.path.join(basePath, (x + '-model'))
    if x == 'none':
        pathTest = testPath
    else:
        pathTest = os.path.join(basePath, (x + '-test'))

    print(pathModel, pathTest)

    tempModelList = create_modellist(pathModel)
    max_accuracy, max_accuracy_model, max_accuracy_pred, tempModelList = predict(pathModel, pathTest, tempModelList, max_accuracy, max_accuracy_model, max_accuracy_pred)

    modelList = pd.concat([modelList, tempModelList])

modelList.to_excel(xlsFile, index = False)

# Sample Data

## Generate Dataset

In [ ]:
batch_size = max_accuracy_model.split('-')[1]
batch_size = int(batch_size)
enhance = max_accuracy_model.split('-')[4]

if enhance == 'none':
    pathModel = os.path.join(basePath, (x + '-model'))
    pathTest = testPath
else:
    pathModel = os.path.join(basePath, (enhance + '-model'))
    pathTest = os.path.join(basePath, (enhance + '-test'))

test_generator, nb_samples, labels = data_generator(pathTest, batch_size)


## Show Confusion Matrix & Classification Report

In [ ]:
cf = confusion_matrix(labels, max_accuracy_pred)
plt.figure(figsize = (25,25))
sns.set(font_scale=1.4)
sns.heatmap(cf, annot=True, cmap="Blues", fmt='g')
# plt.savefig(os.path.join(pathModel, (max_accuracy_model + '-confusion-matrix.png')))

In [ ]:
cr = classification_report(labels, max_accuracy_pred)
print(cr)

# cr.to_excel("classification-report.xlsx", index = False)

## Show Sample Data

In [ ]:
plt.figure(figsize = (25, 25))

start_index = 0
for i in range(25):
    j = random.randint(1, len(labels))
    plt.subplot(5, 5, i + 1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    prediction = max_accuracy_pred[start_index + j]
    actual = labels[start_index + j]
    col = 'g'
    if prediction != actual:
        col = 'r'
    plt.xlabel('Actual={} || Pred={}'.format(actual, prediction), color = col)
    plt.imshow(cv2.imread(os.path.join(test_generator.directory,test_generator.filenames[start_index + j])))
    plt.savefig(os.path.join(pathModel, (max_accuracy_model + '-best-accuracy.png')))
plt.show()